
# 01\_Datasets\_Embeddings\_HF\_Cache\_and\_Stores

## 🤔 Why it matters

* 📚 **Datasets** define truth.
* 🧠 **Embeddings** drive recall.
* 💾 **HF cache** keeps downloads fast & reproducible.
* 🗄️ **Vector stores** make retrieval low-latency at scale.

---

## 📚 Datasets (treat like code)

* 🧱 **Schema**: `id`, `text`, `title?`, `meta{}` , `label?`
* ✂️ **Splits**: `train / val / test` (no leakage!)
* 🧼 **Quality**: dedup, normalize whitespace, strip boilerplate
* 🧯 **PII**: redact/anonymize before embedding
* 🏷️ **Dataset card**: source, license, cleaning steps
* 🧊 **Snapshot**: parquet/JSONL + **hash** (log to MLflow)
* 🎯 **Eval set**: balanced, **adversarial** items included

**Log (MLflow)** → `dataset.id`, `dataset.hash`, `split`, `n_samples` ✅

---

## 🧠 Embeddings (choices that matter)

* 🧩 **Model**: multilingual? domain-tuned? dimension? context len?
* ⚖️ **Trade-off**: recall vs latency vs cost
* 🔣 **Instruction/pooling**: record **instruction text** & pooling method
* 📏 **Preproc**: lowercasing, unicode normalize, stopwords? (be consistent)
* 🧷 **What to embed**: chunk text **+ title + breadcrumbs** (better recall)
* 🔁 **Query vs doc**: sometimes **different** encoders/instructions

**Key metrics** → `hit@k`, `recall`, `MRR/NDCG`, `context_precision`, `diversity@k`
**Log (MLflow)** → `embed.model`, `dim`, `instruction`, `pooling`, `norm=True/False`

---

## 🧵 Chunking (quick guardrails)

* 📐 **Size/overlap**: 500–1000 chars, **10–20% overlap** (start simple)
* 🧭 **Semantic splits** (headings/sentences) > blind fixed splits
* 🏷️ **IDs**: `doc_id`, `chunk_id`, offsets; store in metadata

---

## ⚡ HF Cache (Hugging Face)

* 🗂️ **Locations**:

  * Models → `~/.cache/huggingface/hub`
  * Datasets → `~/.cache/huggingface/datasets`
* 🛠️ **Control**: set `HF_HOME`, `HF_DATASETS_CACHE`, `TRANSFORMERS_CACHE`
* 📌 **Pin revisions**: use **commit SHA** for models/datasets
* ✈️ **Offline mode**: warm cache in CI; avoid cold starts
* 🧹 **Hygiene**: prune old revs; avoid cache explosions

**Log (MLflow)** → `hf.model_ref@sha`, `hf.dataset_ref@sha`

---

## 🗄️ Vector Stores (pick by scale & ops)

| Store                    | Best for           | Notes                                  |
| ------------------------ | ------------------ | -------------------------------------- |
| 🧱 **FAISS**             | Local/small-medium | File-based, super fast, no filters     |
| 🐘 **pgvector**          | SQL + vectors      | Easy ops, good filters, moderate scale |
| 🚀 **Milvus/Zilliz**     | High scale         | HNSW/IVF-PQ, filters, mature           |
| 🍍 **Pinecone/Weaviate** | Managed            | Fast start, pay-per-usage              |
| 🔎 **OpenSearch/ES**     | Hybrid BM25+vec    | Great metadata filtering               |

**Index knobs**

* 🧭 **HNSW**: `M`, `efConstruction`, `efSearch`
* 🧮 **IVF-PQ**: `nlist`, `nprobe`, codebooks
* 🎛️ **Filters**: tags/date/tenant; plan **hybrid** (BM25 + vec) + **reranker**

**Ops metrics** → QPS, p95 latency, recall\@k on canary set, index size, RAM, build time

**Log (MLflow)** → `index.type`, `index.params`, `store.uri`, `index.fingerprint`

---

## 🧰 Caching for \$\$ & speed

* 🔁 **Query embedding cache** (LRU/Redis)
* ♻️ **Provider cache** (if available) for repeated prompts
* 📦 **Chunk dedup**: hash chunks; skip re-embedding unchanged text

---

## 🔐 Governance & deletion

* 📝 **Lineage**: dataset → embed model → index version
* 🧨 **Right to forget**: tombstones + reindex plan
* 🔒 **Access**: per-tenant namespaces; encrypt at rest/in-flight

---

## ✅ Minimal “what to log” (per build)

* 🏷️ `dataset.id/hash`, `split`, `n`
* 🧠 `embed.model`, `dim`, `instruction`, `pooling`, `normalize`
* ✂️ `chunk.size/overlap`, `splitter`
* 🗄️ `store.type/uri`, `index.type/params`, `index.fingerprint`
* 📊 `hit@k`, `recall`, `latency_ms_p95`, `size_gb`, `build_min`

---

## ⚠️ Common pitfalls

* 🌀 Mixing embed models/dims across runs → broken cosine sims
* 🧪 Changing eval set mid-comparison
* 🧷 Forgetting **instruction text** (hidden drift!)
* 🗂️ No delete strategy → compliance risk
* 🧼 Embedding raw PII/secrets

---

## 🚀 Quick wins

* 🧊 Freeze **parquet snapshot + hash**; log to MLflow
* 🧱 Embed **title + path** with text for better recall
* 🧮 Track **context\_use\_rate** (retrieved docs actually used?)
* 🔁 Cache query embeddings; cap `k`; rerank before LLM to cut tokens
* 📌 Pin HF **commit SHAs**; pre-warm caches in CI

---

## 🗣️ One-liner

**“Version your data, pin your embeddings, warm your HF caches, and fingerprint your index—so retrieval stays fast, cheap, and reproducible.”**
